In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/heart-disease-prediction-using-logistic-regression/framingham.csv')

In [ ]:
df.info()

In [ ]:
df.describe(include='all')

In [ ]:
df.isnull().sum()

We notice that there are nulls in columns. Lets replace them with mean, median and mode of a particular variable.

The way to input data in NULL Value:

Delete data with complete.cases () if the amount of NULL Value data is not too much

Replace the data with mean () if the data is numeric and there is no outlier

Replace the data with median () if the data is numeric and there is outlier

Replace data with mode(modus) if the data is factorial orstring and the distribution of variations in the data varies. The table () function can help with the mode(modus) of data

For simplicity we will drop all null containing rows. The result is a dataset reduced by 12%


In [ ]:
data=df#.drop(['education'],axis=1)
data.dropna(axis=0,inplace=True)

data.describe(include='all')

For simplicity we will drop education and all null containing columns. The result is a dataset reduced by 12%


In [ ]:
category=['male','currentSmoker','BPMeds','prevalentStroke','prevalentHyp','diabetes','education']
cols=list(data.columns)
cols.remove('TenYearCHD')
numeric=list(set(cols)-set(category))

Lets do some Chisquared test for independance between categorical variables and predictor variables

In [ ]:
for i in category:
    print("-------"+i+"-------")
    contingency_table=pd.crosstab(data[i],data["TenYearCHD"])
    #print('contingency_table :-\n',contingency_table)

    Observed_Values = contingency_table.values 
    #print("Observed Values :-\n",Observed_Values)

    #Expected Values
    import scipy.stats
    b=scipy.stats.chi2_contingency(contingency_table)
    Expected_Values = b[3]
    #print("Expected Values :-\n",Expected_Values)

    deg_freedom=b[2]
    print("Degree of Freedom:-",deg_freedom)

    from scipy.stats import chi2
    chi_square=sum([(o-e)**2./e for o,e in zip(Observed_Values,Expected_Values)])
    chi_square_statistic=chi_square[0]+chi_square[1]
    print("chi-square statistic:-",chi_square_statistic)

    p_value=1-chi2.cdf(x=chi_square_statistic,df=deg_freedom)
    print('p-value:',p_value)

    if p_value<=0.05:
        print("There is a relationship")
    else:
        print("There is no relationship ")

From the above results. All categorical variables except currSmoker has a significant relationship with Heart Disease

In [ ]:
from scipy import stats
for i in numeric:
    print("------"+i+"--------")
    df_anova = data[["TenYearCHD",i]]

    grps = pd.unique(df_anova["TenYearCHD"].values)
    d_data = {grp:df_anova[i][df_anova["TenYearCHD"] == grp] for grp in grps}

    F, p = stats.f_oneway(d_data[0], d_data[1])
    print("p-value for significance is: ", round(p,4))
    if p<0.05:
        print("reject null hypothesis")
    else:
        print("accept null hypothesis")

Heart rate shows no relation with Heart disease.
Lets see the variation inflation factor for Numeric data

Lets take two approaches here.A model with the above filtered variables. One with feature selection by backward elimination.

Case 1 (backward elimination)

We feed the dataset without scrutinizing the independant variable and apply backward elimination using p-value to come with a set of variables for which p values are below (0.05)

In [ ]:
from statsmodels.tools import add_constant
from statsmodels.discrete.discrete_model import Logit
x=data[cols]
x = add_constant(x)
y=data.TenYearCHD
model=Logit(y,x)
result=model.fit()
result.summary()

The message box shows it took 10 iterations to fit the best possible model and also shows current function value (objective function). This value is helpful when after certain number of iterations the model wont learn so the objective function remains the same

Maximum Likelihood estimation: To find an optimal way to fit a distribution on the data. the optimal place at which the distribution must be placed to define the data well. Bigger the likelihood higher the probability that our model is correct.

Log likely hood- its almost negative always. Bigger the better
LL null- log likely hood null	- log likely hood when there is no independant variable. Aka. useless model

The comparison between these two metrics gives 
LLR p-value- show how statistically significant the model is compared to LL-NULL

Pseudo R squared(McFadden’s R-squared) - somewhere between 0.2 and 0.4 good. Useful only to comapare different variation of same model

In [ ]:
cols=data.columns[:-1]
print(cols)

In [ ]:

def back_feature_elem (data_frame,dep_var,col_list):
    while len(col_list)>0 :
        x=data[col_list]
        x = add_constant(x)
        y=data.TenYearCHD
        model=Logit(y,x)
        result=model.fit(disp=0)
        largest_pvalue=round(result.pvalues,3).nlargest(1)
        if largest_pvalue[0]<(0.05):
            return result
            break
        else:
            col_list=col_list.drop(largest_pvalue.index)
    
result=back_feature_elem(data,data.TenYearCHD,cols)
result.summary()

Feature Selection: Backward elemination (P-value approach)
Takes in the dataframe, the dependent variable and a list of column names, 
runs the regression repeatedly eliminating feature with the highest
P-value above alpha one at a time and returns the regression summary 
with all p-values below alpha

Lets interpret the coefficients of the variables.
The exponential off the coeffcients give the odds ratio.

In [ ]:
print(np.exp(result.params))

the values mean that
for males there is a 75% (1.75-1 =0.75) higher chances of heart disease than for females

similarly,
for 1 year increase in age there is 6.8% higher chance of getting heart disease.

and so on..

Lets do some predictions

In [ ]:
#lets get the confusion matrix based on statsmodel

conf_sm=pd.DataFrame(result.pred_table())
conf_sm.columns=['predicted 0','predicted 1']
conf_sm=conf_sm.rename(index={0:'Actual 0',1:'Actual 1'})
conf_sm

In [ ]:
#train split

import sklearn
new_features=data[['age','male','cigsPerDay','totChol','sysBP','glucose','TenYearCHD']]
x=new_features.iloc[:,:-1]
y=new_features.iloc[:,-1]
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.20,random_state=5)

from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression()
logreg.fit(x_train,y_train)
y_pred=logreg.predict(x_test)


from sklearn import metrics
confusion_matrix = metrics.confusion_matrix(y_test,y_pred)

conf=pd.DataFrame(confusion_matrix)
conf.columns=['predicted 0','predicted 1']
conf=conf.rename(index={0:'Actual 0',1:'Actual 1'})
print(conf)

print("model accuracy:")
print(sklearn.metrics.accuracy_score(y_test,y_pred))

The model predicts the correct output 86.3% of the time


Case 2-

Lets try for model with feature selection done manually. Based on our hypothesis testing. We rejected currentSmoker and heart rate to be of significant predictor.

In [ ]:
x=data.drop(['TenYearCHD','currentSmoker','heartRate'],axis=1)
x=add_constant(x)
y=data.TenYearCHD
model=Logit(y,x)
result=model.fit(disp=0)
result.summary()

Removing all insignificant variables

In [ ]:
x=data.drop(['TenYearCHD','currentSmoker','heartRate','glucose','prevalentStroke','education','BPMeds','prevalentHyp','diabetes','diaBP','BMI'],axis=1)
x=add_constant(x)
y=data.TenYearCHD
model=Logit(y,x)
result=model.fit(disp=0)
result.summary()

In [ ]:
#train split

import sklearn
new_features=data[['age','male','cigsPerDay','totChol','sysBP','TenYearCHD']]
x=new_features.iloc[:,:-1]
y=new_features.iloc[:,-1]
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.20,random_state=5)
#print(x_test)
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression()
logreg.fit(x_train,y_train)
y_pred=logreg.predict(x_test)

from sklearn import metrics
confusion_matrix = metrics.confusion_matrix(y_test,y_pred)

conf=pd.DataFrame(confusion_matrix)
conf.columns=['predicted 0','predicted 1']
conf=conf.rename(index={0:'Actual 0',1:'Actual 1'})
print(conf)

print("model accuracy:")
print(sklearn.metrics.accuracy_score(y_test,y_pred))

There is only slight difference in accuracy between the two models.